In [1]:
## set up imports of local module code
import os
nb_path = os.path.abspath("")
from sys import path
from os.path import dirname

path.insert(0,  dirname(nb_path))

nb_path


'c:\\Users\\ethan\\coding_projects\\pypreql\\experiments'

In [2]:
### Test


from preql import Executor, Dialects
from sqlalchemy import create_engine
from preql.hooks.query_debugger import DebuggingHook
from preql.constants import logger
from logging import StreamHandler
engine = create_engine(r"duckdb:///:memory:", future=True)

test = Executor(engine=engine, dialect=Dialects.DUCK_DB)



In [3]:
import pandas as pd

df = pd.read_csv('./train.csv')

In [4]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.execute_raw_sql("CREATE TABLE raw_titanic AS SELECT * FROM df")


In [6]:
from preql.core.models import Datasource, Concept, ColumnAssignment
from preql.core.enums import DataType, Purpose
env = test.environment
namespace = 'passenger'
id = Concept(name='id', namespace=namespace, datatype=DataType.INTEGER,
                        purpose=Purpose.KEY)
age = Concept(name='age', namespace=namespace, datatype=DataType.INTEGER, purpose=Purpose.PROPERTY)

name = Concept(name='name', namespace=namespace, datatype=DataType.STRING, purpose=Purpose.PROPERTY)

pclass = Concept(name='passenger_class', namespace=namespace, purpose = Purpose.PROPERTY,
                        datatype=DataType.INTEGER
                        )
survived = Concept(name='survived', namespace=namespace, purpose = Purpose.PROPERTY,
                        datatype=DataType.BOOL
                        )
fare = Concept(name='fare', namespace=namespace, purpose = Purpose.PROPERTY, datatype=DataType.FLOAT )
for x in [id, age, survived, name, pclass, fare]:
    env.add_concept(x)

test.environment.add_datasource(
    Datasource(
        identifier = 'raw_data',
        address = 'raw_titanic',
        columns= [ColumnAssignment('passengerid', id ),
                  ColumnAssignment('age', age),
                  ColumnAssignment('survived', survived),
                  ColumnAssignment('pclass', pclass),
                  ColumnAssignment('name', name),
                  ColumnAssignment('fare', fare)]

    ), 

)

Datasource(identifier='raw_data', columns=[ColumnAssignment(alias='passengerid', concept=Concept(name='id', datatype=<DataType.INTEGER: 'int'>, purpose=<Purpose.KEY: 'key'>, metadata=Metadata(description=None, line_number=None, concept_source=<ConceptSource.MANUAL: 'manual'>), lineage=None, namespace='passenger', keys=None, grain=Grain(components=[Concept(name='id', datatype=<DataType.INTEGER: 'int'>, purpose=<Purpose.KEY: 'key'>, metadata=Metadata(description=None, line_number=None, concept_source=<ConceptSource.MANUAL: 'manual'>), lineage=None, namespace='passenger', keys=None, grain=Grain(components=[Concept(name='id', datatype=<DataType.INTEGER: 'int'>, purpose=<Purpose.KEY: 'key'>, metadata=Metadata(description=None, line_number=None, concept_source=<ConceptSource.MANUAL: 'manual'>), lineage=None, namespace='passenger', keys=None, grain=Grain(components=[], nested=False))], nested=True))], nested=False)), modifiers=[]), ColumnAssignment(alias='age', concept=Concept(name='age', dat

In [7]:
results = test.execute_text('select passenger.survived, count(passenger.id)->count_survived;')

for row in results[0].fetchall():
    print(row)

(0, 549)
(1, 342)


In [8]:
families = test.execute_text("select passenger.name limit 100;")

# query = test.generate_sql("select split(passenger.name, ',')[0]-> passenger.family;")

# print(query[0])
for row in families[0].fetchall():
    print(row)

('Braund, Mr. Owen Harris',)
('Cumings, Mrs. John Bradley (Florence Briggs Thayer)',)
('Heikkinen, Miss. Laina',)
('Futrelle, Mrs. Jacques Heath (Lily May Peel)',)
('Allen, Mr. William Henry',)
('Moran, Mr. James',)
('McCarthy, Mr. Timothy J',)
('Palsson, Master. Gosta Leonard',)
('Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',)
('Nasser, Mrs. Nicholas (Adele Achem)',)
('Sandstrom, Miss. Marguerite Rut',)
('Bonnell, Miss. Elizabeth',)
('Saundercock, Mr. William Henry',)
('Andersson, Mr. Anders Johan',)
('Vestrom, Miss. Hulda Amanda Adolfina',)
('Hewlett, Mrs. (Mary D Kingcome) ',)
('Rice, Master. Eugene',)
('Williams, Mr. Charles Eugene',)
('Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)',)
('Masselmani, Mrs. Fatima',)
('Fynney, Mr. Joseph J',)
('Beesley, Mr. Lawrence',)
('McGowan, Miss. Anna "Annie"',)
('Sloper, Mr. William Thompson',)
('Palsson, Miss. Torborg Danira',)
('Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)',)
('Emir, Mr. Farred Chehab',)
('Fortune,

In [9]:
results = test.execute_raw_sql('''
SELECT
    local_raw_data."name" as "passenger_name",
     STRING_SPLIT( local_raw_data."name" , ',' )[1] as split_name
FROM
    raw_titanic as local_raw_data

''')
for r in results.fetchall():
    print(r)

('Braund, Mr. Owen Harris', 'Braund')
('Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'Cumings')
('Heikkinen, Miss. Laina', 'Heikkinen')
('Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'Futrelle')
('Allen, Mr. William Henry', 'Allen')
('Moran, Mr. James', 'Moran')
('McCarthy, Mr. Timothy J', 'McCarthy')
('Palsson, Master. Gosta Leonard', 'Palsson')
('Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', 'Johnson')
('Nasser, Mrs. Nicholas (Adele Achem)', 'Nasser')
('Sandstrom, Miss. Marguerite Rut', 'Sandstrom')
('Bonnell, Miss. Elizabeth', 'Bonnell')
('Saundercock, Mr. William Henry', 'Saundercock')
('Andersson, Mr. Anders Johan', 'Andersson')
('Vestrom, Miss. Hulda Amanda Adolfina', 'Vestrom')
('Hewlett, Mrs. (Mary D Kingcome) ', 'Hewlett')
('Rice, Master. Eugene', 'Rice')
('Williams, Mr. Charles Eugene', 'Williams')
('Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)', 'Vander Planke')
('Masselmani, Mrs. Fatima', 'Masselmani')
('Fynney, Mr. Joseph J', 'Fynney')
('Beesley

In [10]:

test.hooks= [DebuggingHook()]
x= (test.generate_sql("""select split(passenger.name, ',')[1] -> passenger.family, 
                                   passenger.id.count,
                                   sum(passenger.fare) -> total_family_fare
                                    order by passenger.id.count desc;""")[0])
print(x)
families_split = test.execute_text("""select split(passenger.name, ',')[1] -> passenger.family, 
                                   passenger.id.count,
                                   sum(passenger.fare) -> total_family_fare
                                    order by passenger.id.count desc;""")

for row in families_split[0].fetchall():
    print(row)

[QUERY BUILD] getting source datasource for query with output ['passenger.family<passenger.family>', 'passenger.id.count<>', 'local.total_family_fare<passenger.family>']


grain: Grain<passenger.family>


[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.family<passenger.family>', 'passenger.id.count<>', 'local.total_family_fare<passenger.family>']
	[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.id<passenger.id>', 'passenger.family<passenger.family>']
	[CONCEPT DETAIL - SELECT NODE] found direct select from Address(location='raw_titanic')
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.id', 'passenger.family'] from [SelectNode<passenger.id>], have ['passenger.id']
		[CONCEPT DETAIL] Beginning sourcing loop for ['local._anon_function_input_7255585570204832<>', 'passenger.id<passenger.id>']
			[CONCEPT DETAIL - SELECT NODE] found direct select from Address(location='raw_titanic')
			[CONCEPT DETAIL] found direct select node with all 2 concepts, returning static selection
		[CONCEPT DETAIL] finished a loop iteration looking for ['local._anon_function_input_7255585570204832', 'passenger.id'] from [SelectNode<local._anon_function_input_7255585570204832,pa

		[CONCEPT DETAIL - MERGE NODE] Merge node has only one parent with the same outputs as this merge node, dropping merge node 
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.fare', 'passenger.family'] from [SelectNode<passenger.fare>, SelectNode<passenger.family,passenger.fare>], have ['passenger.fare', 'passenger.fare', 'passenger.family', 'passenger.fare']
	[CONCEPT DETAIL] have all concepts, have ['passenger.fare', 'passenger.family'] from [SelectNode<passenger.fare>, SelectNode<passenger.family,passenger.fare>] checking for single connected graph
	[CONCEPT DETAIL] Graph analysis: 1 subgraphs found
	[CONCEPT DETAIL] One fully connected subgraph returned, sourcing ['passenger.fare', 'passenger.family'] successful.
	[CONCEPT DETAIL - MERGE NODE] Merge node not required as one parent node has all required output properties
[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.family', 'passenger.id.count', 'local.total_family_fare'] from [GroupNode<pass

GroupNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
  MergeNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
    GroupNode<passenger.family,passenger.id.count>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.id>->Grain<passenger.id>
        SelectNode<passenger.id>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.id>->Grain<passenger.id>
          MergeNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
            SelectNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
              StaticSelectNode<passenger.id,passenger.name>->Grain<passenger.id>
    GroupNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.fare>->Grain<passenger.id>
        SelectNode<passenger.fare>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.fa

[QUERY BUILD] getting source datasource for query with output ['passenger.family<passenger.family>', 'passenger.id.count<>', 'local.total_family_fare<passenger.family>']
[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.family<passenger.family>', 'passenger.id.count<>', 'local.total_family_fare<passenger.family>']
	[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.id<passenger.id>', 'passenger.family<passenger.family>']
	[CONCEPT DETAIL - SELECT NODE] found direct select from Address(location='raw_titanic')
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.id', 'passenger.family'] from [SelectNode<passenger.id>], have ['passenger.id']
		[CONCEPT DETAIL] Beginning sourcing loop for ['local._anon_function_input_7255585570204832<>', 'passenger.id<passenger.id>']
			[CONCEPT DETAIL - SELECT NODE] found direct select from Address(location='raw_titanic')
			[CONCEPT DETAIL] found direct select node with all 2 concepts, returning static selection
		[CONCEPT DET

grain: Grain<passenger.family>
GroupNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
  MergeNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
    GroupNode<passenger.family,passenger.id.count>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.id>->Grain<passenger.id>
        SelectNode<passenger.id>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.id>->Grain<passenger.id>
          MergeNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
            SelectNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
              StaticSelectNode<passenger.id,passenger.name>->Grain<passenger.id>
    GroupNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.fare>->Grain<passenger.id>
        SelectNode<passenger.fare>->Grain<passenger.id>
        SelectNod

      cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508.`_anon_function_input_7255585570204832`[1] as `passenger_family`,
      cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508.`passenger_id` as `passenger_id`,
      cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508.`passenger_fare` as `passenger_fare`
  FROM
      cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 as cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508
  
[RENDERING] [local._anon_function_input_7255585570204832] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 alias=True
[RENDERING] [local._anon_function_input_7255585570204832] rendering lineage concept
[RENDERING] [passenger.name] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 alias=False
[RENDERING] [passenger.name] Basic reference, using source address for passenger.name
[RENDERING] [local._anon_function_input_7255585570204832] ignori

-----
     cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 -> False -> ['local._anon_function_input_7255585570204832', 'passenger.id', 'passenger.fare']
-----
       cte_raw_data_at_passenger_id_8958832269400449 -> False -> ['passenger.name', 'passenger.id', 'passenger.fare']
-----
('Andersson', 9, 234.6458)
('Sage', 7, 486.85)
('Johnson', 6, 41.1749)
('Panula', 6, 238.125)
('Goodwin', 6, 281.4)
('Skoog', 6, 167.4)
('Carter', 6, 532.0)
('Rice', 5, 145.625)
('Palsson', 4, 84.3)
('Williams', 4, 98.5292)
('Asplund', 4, 125.55)
('Fortune', 4, 1052.0)
('Harper', 4, 219.4584)
('Harris', 4, 187.95)
('Ford', 4, 137.5)
('Gustafsson', 4, 33.4708)
('Lefebre', 4, 101.8668)
('Brown', 4, 118.7208)
('Kelly', 4, 37.05)
('Hart', 4, 85.6083)
('Baclini', 4, 77.0332)
('Smith', 4, 77.44579999999999)
('Moran', 3, 56.7583)
('Vander Planke', 3, 54.0)
('Meyer', 3, 177.3416)
('Laroche', 3, 124.7376)
('West', 3, 83.25)
('Jussila', 3, 27.575)
('Hickman', 3, 220.5)
('Boulos', 3, 37.7166)
('Navratil',

In [11]:
ctas = test.execute_text("""
PERSIST family_info INTO family_info FROM                             
SELECT
    split(passenger.name, ',')[1] -> passenger.family, 
    passenger.id.count,
    sum(passenger.fare) -> total_family_fare
;
                                   """)

for row in ctas[0].fetchall():
    print(row)

Persist statement persisting to Address(location='family_info')

[QUERY BUILD] getting source datasource for query with output ['passenger.family<passenger.family>', 'passenger.id.count<>', 'local.total_family_fare<passenger.family>']
[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.family<passenger.family>', 'passenger.id.count<>', 'local.total_family_fare<passenger.family>']
	[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.id<passenger.id>', 'passenger.family<passenger.family>']
	[CONCEPT DETAIL - SELECT NODE] found direct select from Address(location='raw_titanic')
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.id', 'passenger.family'] from [SelectNode<passenger.id>], have ['passenger.id']
		[CONCEPT DETAIL] Beginning sourcing loop for ['local._anon_function_input_7255585570204832<>', 'passenger.id<passenger.id>']
			[CONCEPT DETAIL - SELECT NODE] found direct select from Address(location='raw_titanic')
			[CONCEPT DETAIL] found direct select node with all 2 concepts, returning static selection
		[CONCEPT DET


grain: Grain<passenger.family>


[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.family', 'passenger.id.count', 'local.total_family_fare'] from [GroupNode<passenger.family,passenger.id.count>, GroupNode<local.total_family_fare,passenger.family>], have ['passenger.id.count', 'passenger.family', 'passenger.id.count', 'passenger.family', 'local.total_family_fare', 'passenger.family']
[CONCEPT DETAIL] have all concepts, have ['passenger.family', 'passenger.id.count', 'local.total_family_fare'] from [GroupNode<passenger.family,passenger.id.count>, GroupNode<local.total_family_fare,passenger.family>] checking for single connected graph
[CONCEPT DETAIL] Graph analysis: 1 subgraphs found
[CONCEPT DETAIL] One fully connected subgraph returned, sourcing ['passenger.family', 'passenger.id.count', 'local.total_family_fare'] successful.
[CONCEPT DETAIL - MERGE NODE] merging two nodes with raw_data_at_passenger_id_at_passenger_id_at_passenger_id_at_passenger_family
[MODELS] merging raw_data_at_passenger_id_at_pas

GroupNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
  MergeNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
    GroupNode<passenger.family,passenger.id.count>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.id>->Grain<passenger.id>
        SelectNode<passenger.id>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.id>->Grain<passenger.id>
          MergeNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
            SelectNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
              StaticSelectNode<passenger.id,passenger.name>->Grain<passenger.id>
    GroupNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.fare>->Grain<passenger.id>
        SelectNode<passenger.fare>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.fa

[RENDERING] [passenger.id.count] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_at_passenger_family_1099462592324186 alias=True
[RENDERING] [passenger.id.count] rendering lineage concept
[RENDERING] [passenger.id] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_at_passenger_family_1099462592324186 alias=False
[RENDERING] [passenger.id] Basic reference, using source address for passenger.id
[RENDERING] [passenger.family] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_at_passenger_family_1099462592324186 alias=True
[RENDERING] [passenger.name] not found in source map for cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_at_passenger_family_1099462592324186, have cte keys ['passenger.family', 'passenger.id', 'passenger.fare'] and datasource keys ['passenger.family', 'passenger.id', 'passenger.fare']
[RENDERING] [local._anon_function_input_7255585570204832] not found 

(667,)


In [12]:
for name, ds in test.environment.datasources.items():
    for x in ds.concepts:
        print(x)

passenger.id<passenger.id>
passenger.age<>
passenger.survived<>
passenger.passenger_class<>
passenger.name<>
passenger.fare<>
passenger.family<passenger.family>
passenger.id.count<passenger.family>
local.total_family_fare<passenger.family>


In [13]:
from preql.core.query_processor import source_query_concepts


# logger.addHandler(StreamHandler())
source = source_query_concepts([test.environment.concepts['passenger.family'].with_grain(test.environment.concepts['passenger.family'])],
                       test.environment)

DebuggingHook().process_root_strategy_node(source)

[CONCEPT DETAIL - SELECT NODE] found direct select from Address(location='family_info')
[CONCEPT DETAIL] found direct select node with all 1 concepts, returning static selection
[CONCEPT DETAIL - GROUP NODE] Output of group by node equals input of group by node ['passenger.family']
[CONCEPT DETAIL - GROUP NODE] No group by required, returning parent node


GroupNode<passenger.family>->Grain<passenger.family>
  StaticSelectNode<passenger.family>->Grain<passenger.family>


In [14]:
ctas = test.generate_sql(
    'select passenger.family, total_family_fare;'
)

print(ctas[0])

[QUERY BUILD] getting source datasource for query with output ['passenger.family<passenger.family>', 'local.total_family_fare<passenger.family>']
[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.family<passenger.family>', 'local.total_family_fare<passenger.family>']
	[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.fare<>', 'passenger.family<passenger.family>']
		[CONCEPT DETAIL - SELECT NODE] found direct select from Address(location='raw_titanic')


grain: Grain<passenger.family>


		[CONCEPT DETAIL] found direct select node with all 6 concepts, returning static selection
	[CONCEPT DETAIL - MERGE NODE] Merge node has 1 parents, starting merge
	[CONCEPT DETAIL - MERGE NODE] potential merge keys ['passenger.id', 'passenger.age', 'passenger.survived', 'passenger.passenger_class', 'passenger.name', 'passenger.fare'] for raw_data_at_passenger_id
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.fare', 'passenger.family'] from [MergeNode<passenger.family,passenger.fare>], have ['passenger.fare', 'passenger.family']
	[CONCEPT DETAIL] have all concepts, have ['passenger.fare', 'passenger.family'] from [MergeNode<passenger.family,passenger.fare>] checking for single connected graph
	[CONCEPT DETAIL] Graph analysis: 1 subgraphs found
	[CONCEPT DETAIL] One fully connected subgraph returned, sourcing ['passenger.fare', 'passenger.family'] successful.
	[CONCEPT DETAIL - MERGE NODE] Merge node has only one parent with the same outputs as this merge node, drop

[RENDERING] [local.total_family_fare] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_7883406132832648 alias=True
[RENDERING] [local.total_family_fare] rendering lineage concept


GroupNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
  MergeNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
    GroupNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.fare>->Grain<passenger.id>
        MergeNode<passenger.family,passenger.fare>->Grain<passenger.id>
          StaticSelectNode<passenger.age,passenger.fare,passenger.id,passenger.name,passenger.passenger_class,passenger.survived>->Grain<passenger.id>
QueryDatasource<group>->['local.total_family_fare', 'passenger.family']
  QueryDatasource<>->['passenger.fare', 'passenger.family']
    QueryDatasource<>->['passenger.id', 'passenger.age', 'passenger.survived', 'passenger.passenger_class', 'passenger.name', 'passenger.fare']
      Datasource<>->['passenger.id', 'passenger.age', 'passenger.survived', 'passenger.passenger_class', 'passenger.name', 'passenger.fare']
 cte_raw_data_at_passenger_id_at_passenger_id_at_pass

[RENDERING] [passenger.fare] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_7883406132832648 alias=False
[RENDERING] [passenger.fare] Basic reference, using source address for passenger.fare
[RENDERING] [passenger.family] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_7883406132832648 alias=True
[RENDERING] [passenger.name] not found in source map for cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_7883406132832648, have cte keys ['passenger.fare', 'passenger.family'] and datasource keys ['passenger.fare', 'passenger.family']
[RENDERING] [local._anon_function_input_7255585570204832] not found in source map for cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_7883406132832648, have cte keys ['passenger.fare', 'passenger.family'] and datasource keys ['passenger.fare', 'passenger.family']
[RENDERING] [passenger.family] Complex reference falling back to source address
[RENDE

-----
   cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 -> False -> ['passenger.fare', 'passenger.family']
-----
     cte_raw_data_at_passenger_id_8958832269400449 -> False -> ['passenger.id', 'passenger.age', 'passenger.survived', 'passenger.passenger_class', 'passenger.name', 'passenger.fare']
-----


[RENDERING] [passenger.name] not found in source map for cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_7883406132832648, have cte keys ['passenger.fare', 'passenger.family'] and datasource keys ['passenger.fare', 'passenger.family']
[RENDERING] [local._anon_function_input_7255585570204832] not found in source map for cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_7883406132832648, have cte keys ['passenger.fare', 'passenger.family'] and datasource keys ['passenger.fare', 'passenger.family']
[RENDERING] [passenger.family] Complex reference falling back to source address



WITH 
rhash_7192012207602026 as (
SELECT
    local_raw_data."passengerid" as "passenger_id",
    local_raw_data."age" as "passenger_age",
    local_raw_data."survived" as "passenger_survived",
    local_raw_data."pclass" as "passenger_passenger_class",
    local_raw_data."name" as "passenger_name",
    local_raw_data."fare" as "passenger_fare"
FROM
    raw_titanic as local_raw_data
),
rhash_3487714809975377 as (
SELECT
    rhash_7192012207602026."passenger_fare" as "passenger_fare",
    STRING_SPLIT( rhash_7192012207602026."passenger_name" , ',' )[1] as "passenger_family"
FROM
    rhash_7192012207602026 as rhash_7192012207602026
),
rhash_8380470967967938 as (
SELECT
    sum(rhash_3487714809975377."passenger_fare") as "total_family_fare",
    rhash_3487714809975377."passenger_family" as "passenger_family"
FROM
    rhash_3487714809975377 as rhash_3487714809975377

GROUP BY 
    rhash_3487714809975377."passenger_family"
)
SELECT
    rhash_8380470967967938."passenger_family",
    rhash_83

In [20]:
results = test.execute_text('''
 
                           key passenger.family.size <- count(passenger.id) by passenger.family;
                        select 
                            passenger.name,
                            passenger.family, 
                            passenger.family.size,
                            passenger.fare / total_family_fare -> passenger_family_fare_percent
                            ;
    
                            ''')


for row in results[0].fetchall():
    print(row)

[QUERY BUILD] getting source datasource for query with output ['passenger.name<>', 'passenger.family<passenger.family>', 'passenger.family.size<passenger.family>', 'local.passenger_family_fare_percent<passenger.family,passenger.name>']
[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.name<>', 'passenger.family<passenger.family>', 'passenger.family.size<passenger.family>', 'local.passenger_family_fare_percent<passenger.family,passenger.name>']
	[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.id<passenger.id>', 'passenger.family<passenger.family>']


		[CONCEPT DETAIL - SELECT NODE] found direct select from Address(location='raw_titanic')
		[CONCEPT DETAIL] found direct select node with all 6 concepts, returning static selection
	[CONCEPT DETAIL - MERGE NODE] Merge node has 1 parents, starting merge
	[CONCEPT DETAIL - MERGE NODE] potential merge keys ['passenger.id', 'passenger.age', 'passenger.survived', 'passenger.passenger_class', 'passenger.name', 'passenger.fare'] for raw_data_at_passenger_id
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.id', 'passenger.family'] from [MergeNode<passenger.family,passenger.id>], have ['passenger.id', 'passenger.family']
	[CONCEPT DETAIL] have all concepts, have ['passenger.id', 'passenger.family'] from [MergeNode<passenger.family,passenger.id>] checking for single connected graph
	[CONCEPT DETAIL] Graph analysis: 1 subgraphs found
	[CONCEPT DETAIL] One fully connected subgraph returned, sourcing ['passenger.id', 'passenger.family'] successful.
	[CONCEPT DETAIL - MERGE NODE]

grain: Grain<passenger.family,passenger.name>


		[CONCEPT DETAIL - MERGE NODE] potential merge keys ['passenger.id', 'passenger.age', 'passenger.survived', 'passenger.passenger_class', 'passenger.name', 'passenger.fare'] for raw_data_at_passenger_id
		[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.fare', 'passenger.family'] from [MergeNode<passenger.family,passenger.fare>], have ['passenger.fare', 'passenger.family']
		[CONCEPT DETAIL] have all concepts, have ['passenger.fare', 'passenger.family'] from [MergeNode<passenger.family,passenger.fare>] checking for single connected graph
		[CONCEPT DETAIL] Graph analysis: 1 subgraphs found
		[CONCEPT DETAIL] One fully connected subgraph returned, sourcing ['passenger.fare', 'passenger.family'] successful.
		[CONCEPT DETAIL - MERGE NODE] Merge node has only one parent with the same outputs as this merge node, dropping merge node 
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.fare', 'local.total_family_fare', 'passenger.family', 'passenger.name'] f

GroupNode<local.passenger_family_fare_percent,passenger.family,passenger.family.size,passenger.name>->Grain<passenger.family,passenger.name>
  MergeNode<local.passenger_family_fare_percent,passenger.family,passenger.family.size,passenger.name>->Grain<passenger.family,passenger.id>
    GroupNode<passenger.family,passenger.family.size>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.id>->Grain<passenger.id>
        MergeNode<passenger.family,passenger.id>->Grain<passenger.id>
          StaticSelectNode<passenger.age,passenger.fare,passenger.id,passenger.name,passenger.passenger_class,passenger.survived>->Grain<passenger.id>
    MergeNode<passenger.family,passenger.name>->Grain<passenger.id>
      StaticSelectNode<passenger.age,passenger.fare,passenger.id,passenger.name,passenger.passenger_class,passenger.survived>->Grain<passenger.id>
    SelectNode<local.passenger_family_fare_percent,passenger.family,passenger.name>->Grain<passenger.family,passenger.id>
      MergeNo

...

-----
     cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_join_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_passenger_id_at_passenger_family_passenger_id_627491867586781 -> False -> ['local.passenger_family_fare_percent', 'passenger.family', 'passenger.name']
-----
       cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_join_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_passenger_id_2674681277256832 -> False -> ['passenger.fare', 'local.total_family_fare', 'passenger.family', 'passenger.name']
-----
         cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_7883406132832648 -> True -> ['local.total_family_fare', 'passenger.family']
-----
           cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 -> False -> ['passenger.fare', 'passenger.family']
-----
             cte_raw_data_at_passenger_id_8958832269400449 -> False -> ['passenger.id', 'passenger.age', 'passenger.survived', 'passenger.passen

[RENDERING] [passenger.fare] Basic reference, using source address for passenger.fare
            
            SELECT
                local_raw_data.`passengerid` as `passenger_id`,
                local_raw_data.`age` as `passenger_age`,
                local_raw_data.`survived` as `passenger_survived`,
                local_raw_data.`pclass` as `passenger_passenger_class`,
                local_raw_data.`name` as `passenger_name`,
                local_raw_data.`fare` as `passenger_fare`
            FROM
                raw_titanic as local_raw_data
            
[RENDERING] [passenger.fare] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 alias=True
[RENDERING] [passenger.fare] Basic reference, using source address for passenger.fare
[RENDERING] [passenger.family] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 alias=True
[RENDERING] [passenger.family] rendering lineage concept
[RENDERING] [local._anon_functio

-----
         cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 -> False -> ['passenger.fare', 'passenger.family', 'passenger.name']
-----
           cte_raw_data_at_passenger_id_8958832269400449 -> False -> ['passenger.id', 'passenger.age', 'passenger.survived', 'passenger.passenger_class', 'passenger.name', 'passenger.fare']
-----


[RENDERING] [local._anon_function_input_7255585570204832] not found in source map for cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_7883406132832648, have cte keys ['passenger.fare', 'passenger.family', 'passenger.id'] and datasource keys ['passenger.fare', 'passenger.family']
[RENDERING] [passenger.family] Complex reference falling back to source address
[RENDERING] [passenger.fare] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_join_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_passenger_id_2674681277256832 alias=True
[RENDERING] [passenger.fare] Basic reference, using source address for passenger.fare
[RENDERING] [local.total_family_fare] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_join_raw_data_at_passenger_id_at_passenger_id_at_passenger_family_passenger_id_2674681277256832 alias=True
[RENDERING] [local.total_family_fare] Complex reference falling back to source

('Braund, Mr. Lewis Richard', 'Braund', 2, 0.49285804222219115)
('Heikkinen, Miss. Laina', 'Heikkinen', 1, 1.0)
('Palsson, Mrs. Nils (Alma Cornelia Berglund)', 'Palsson', 4, 0.25)
('Johnson, Master. Harold Theodor', 'Johnson', 6, 0.2703904563216911)
('Nasser, Mr. Nicholas', 'Nasser', 2, 0.5)
('Bonnell, Miss. Elizabeth', 'Bonnell', 1, 1.0)
('Hewlett, Mrs. (Mary D Kingcome) ', 'Hewlett', 1, 1.0)
('Rice, Mrs. William (Margaret Norton)', 'Rice', 5, 0.2)
('Williams, Mr. Leslie', 'Williams', 4, 0.16340333626985706)
('Vander Planke, Mr. Leo Edmondus', 'Vander Planke', 3, 0.3333333333333333)
('Masselmani, Mrs. Fatima', 'Masselmani', 1, 1.0)
('Fynney, Mr. Joseph J', 'Fynney', 1, 1.0)
('Beesley, Mr. Lawrence', 'Beesley', 1, 1.0)
('McGowan, Miss. Anna "Annie"', 'McGowan', 1, 1.0)
('Emir, Mr. Farred Chehab', 'Emir', 1, 1.0)
('Fortune, Mr. Mark', 'Fortune', 4, 0.25)
('Uruchurtu, Don. Manuel E', 'Uruchurtu', 1, 1.0)
('Spencer, Mrs. William Augustus (Marie Eugenie)', 'Spencer', 1, 1.0)
('Meyer, Mr. A